In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(Market Beat or Market Meet)

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [6]:
tickers_file_name = "Tickers_Example.csv"
tickers_file_original = pd.read_csv(tickers_file_path , header = None, names=['Ticker'])

start_date = "2023-10-02"
end_date = "2024-09-30"
drop_tickers = []

for tick in tickers_file_original['Ticker']:    

    try:

        ticker = yf.Ticker(tick) 

        if ticker.info['quoteType'] != 'EQUITY':
            drop_tickers.append(tick)
            continue
                
        if ticker.fast_info.get('currency') not in ['USD', 'CAD']:
            drop_tickers.append(tick)
            continue

        history = ticker.history(start = start_date, end = end_date)

        trading_days = history['Volume'].resample('ME').count()
        volume = history['Volume'].resample('ME').sum()
        trading_days = trading_days[trading_days>=18]
        volume = volume[volume.index==trading_days.index]
        monthly_volume = volume.mean()

        if monthly_volume<100000: 
            drop_tickers.append(tick)

    except:
            drop_tickers.append(tick)
            pass
    
tickers_file_original.drop(tickers_file_original[tickers_file_original["Ticker"].isin(drop_tickers)].index, inplace=True)
tickers_file_original = tickers_file_original.drop_duplicates(subset=['Ticker'], keep='last')
tickers_file_original = tickers_file_original.reset_index(drop=True)
tickers_file_original.index = tickers_file_original.index + 1
# print(tickers_file_original)

['AAPL', 'ABBV', 'ABT', 'ACN', 'AGN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BB.TO', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CELG', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTN', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


In [7]:
tickers_file_original

,Ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AIG
5,AMZN
6,AXP
7,BA
8,BAC
9,BB.TO


In [8]:
sp500 = yf.Ticker("^GSPC").history(start=start_date, end=end_date)['Close']
sp500.index = sp500.index.date

tsx60 = yf.Ticker("XIU.TO").history(start=start_date, end=end_date)['Close']
tsx60.index = tsx60.index.date

total_investment = 1000000
sp500val = (total_investment / sp500.iloc[0]) * sp500.iloc[-1]
sp500val = round(sp500val,2)
# sp500val

tsx60val = (total_investment / tsx60.iloc[0]) * tsx60.iloc[-1]
tsx60val = round(tsx60val,2)
# tsx60val

average = (sp500val + tsx60val) / 2
average = round(average,2)
average

1311957.2

In [16]:
# take exchange rates from yfinance
exchange = 'USDCAD=X'
exchange_ticker = yf.Ticker(exchange)
exchange_rate = exchange_ticker.history(interval = '1mo', start=start_date, end=end_date)["Close"]
exchange_rate = pd.DataFrame(exchange_rate)
exchange_rate.index = exchange_rate.index.strftime('%Y-%m-%d')

In [ ]:
# convert everything to CAD !!!!!!

In [24]:
sp500 = yf.Ticker("^GSPC").history(start=start_date, end=end_date)['Close'] 
sp500.index = sp500.index.date
sp500_return = sp500.pct_change().dropna()
tsx60 = yf.Ticker("XIU.TO").history(start=start_date, end=end_date)['Close']
tsx60.index = tsx60.index.date
tsx60_return = tsx60.pct_change().dropna()

common_dates = sp500_return.index.intersection(tsx60_return.index)

average_df = pd.DataFrame(data = {"% Return" : (sp500_return.loc[common_dates] + tsx60_return.loc[common_dates])/2}, index = common_dates)
average_df

,% Return
2023-10-03,-0.011329
2023-10-04,0.004747
2023-10-05,0.002284
2023-10-06,0.008140
2023-10-10,0.009783
...,...
2024-09-23,0.002371
2024-09-24,0.001393
2024-09-25,-0.001895
2024-09-26,0.004643


In [26]:
stocks = {key: 0 for key in tickers_file_original['Ticker'].values}
current_data = pd.DataFrame()
metrics = pd.DataFrame(index=list(stocks.keys()), columns=['SP-beta','SP-corr','TSX-beta','TSX-corr', 'S&P500', 'TSX60'])

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_df = pd.read_html(url)[0]
sp500companies = sp500_df.Symbol.tolist()

tsx60companies = ["RY.TO", "TD.TO", "SHOP.TO", "ENB.TO", "CNR.TO", "SU.TO", "BN.TO", "CSU.TO", 
    "MFC.TO", "TRP.TO", "BMO.TO", "BNS.TO", "ATD.TO", "ABX.TO", "BCE.TO", "CM.TO", 
    "CP.TO", "NTR.TO", "TRI.TO", "WCN.TO", "MG.TO", "L.TO", "IFC.TO", "WN.TO", 
    "SLF.TO", "FNV.TO", "TECK.B.TO", "DOL.TO", "OTEX.TO", "FM.TO", "IMO.TO", 
    "CVE.TO", "PPL.TO", "CNQ.TO", "FTS.TO", "H.TO", "RCI.B.TO", "T.TO", "AQN.TO", 
    "SAP.TO", "MRU.TO", "QSR.TO", "WPM.TO", "POW.TO", "EMA.TO", "AEM.TO", 
    "GIL.TO", "GIB.A.TO", "TOU.TO", "BEP.UN.TO", "BIP.UN.TO", "CTC.A.TO", 
    "CCL.B.TO", "K.TO", "WSP.TO", "IFP.TO"]

def in_sp500(ticker, sp500companies):
    if ticker in sp500companies:
        return 5
    else:
        return 0
    
def in_tsx60(ticker, tsx60companies):
    if ticker in tsx60companies:
        return 5
    else: 
        return 0

    
sp500_returns = sp500.pct_change(fill_method=None)
tsx60_returns = tsx60.pct_change(fill_method=None)

for i in stocks:
    
    current_data[i] = yf.Ticker(i).history(start=start_date, end=end_date)["Close"]
    metrics.loc[i, 'S&P500'] = in_sp500(i, sp500companies)
    metrics.loc[i, 'TSX60'] = in_sp500(i, tsx60companies)
    
    stock_returns = current_data[i].pct_change().dropna()
    stock_returns.index = stock_returns.index.date
    
    common_dates = stock_returns.index.intersection(sp500_returns.index).intersection(tsx60_returns.index)

    stock_returns = stock_returns.loc[common_dates]
    aligned_sp500_returns = sp500_returns.loc[common_dates]
    aligned_tsx60_returns = tsx60_returns.loc[common_dates]
    
    covariance = np.cov(stock_returns, aligned_sp500_returns)[0, 1]
    variance = np.var(aligned_sp500_returns)
    beta_sp500 = covariance / variance
    
    correlation_sp500 = np.corrcoef(stock_returns, aligned_sp500_returns)[0, 1]
    
    covariance_tsx = np.cov(stock_returns, aligned_tsx60_returns)[0, 1]
    variance_tsx = np.var(aligned_tsx60_returns)
    beta_tsx60 = covariance_tsx / variance_tsx
    
    correlation_tsx60 = np.corrcoef(stock_returns, aligned_tsx60_returns)[0, 1]
    
    metrics.loc[i, 'SP-corr'] = correlation_sp500
    metrics.loc[i, 'SP-beta'] = beta_sp500
    metrics.loc[i, 'TSX-corr'] = correlation_tsx60
    metrics.loc[i, 'TSX-beta'] = beta_tsx60

    # stocks[i] = round(100 * (correlation_sp500 + beta_sp500 + beta_tsx60 + correlation_tsx60),2)


metrics

,SP-beta,SP-corr,TSX-beta,TSX-corr,S&P500,TSX60
AAPL,0.947255,0.534552,0.465565,0.235358,5,0
ABBV,0.20373,0.132373,0.23493,0.136745,5,0
ABT,0.114747,0.075522,0.216025,0.127368,5,0
ACN,0.715191,0.381639,0.665242,0.318007,5,0
AIG,0.743347,0.456687,0.899801,0.495221,5,0
AMZN,1.515451,0.664044,0.936852,0.367749,5,0
AXP,1.021987,0.537694,1.148117,0.54113,5,0
BA,0.907224,0.334471,1.079023,0.35637,5,0
BAC,0.859954,0.442247,1.319852,0.608053,5,0
BB.TO,1.35957,0.307291,1.455341,0.294672,0,0


In [27]:
beta_threshold = 0.1  
penalty_weight = 20  

ranked_stocks = {}
for ticker, score in stocks.items():
    sp_beta = metrics.loc[ticker, 'SP-beta']
    tsx_beta = metrics.loc[ticker, 'TSX-beta']
    in_index = metrics.loc[ticker, 'S&P500'] + metrics.loc[ticker, 'TSX60']
    
    sp_penalty = abs(sp_beta - 1) * penalty_weight
    tsx_penalty = abs(tsx_beta - 1) * penalty_weight

    adjusted_score = score - (sp_penalty + tsx_penalty) + in_index
    ranked_stocks[ticker] = adjusted_score

ranked_stocks = dict(sorted(ranked_stocks.items(), key=lambda item: item[1], reverse=True))

stocks = dict(list(ranked_stocks.items())[:24])

stocks

{'BLK': 3.2832783787531428,
 'AXP': 1.5979176280644491,
 'BA': 1.564027385667821,
 'UPS': -0.11842143461484333,
 'C': -1.2792347188543873,
 'AIG': -2.1370411210063445,
 'BK': -2.174603919197061,
 'BAC': -4.197967888623985,
 'BIIB': -4.854892477744912,
 'LLY': -6.1860765877841075,
 'AMZN': -6.571982993963056,
 'AAPL': -6.743602766489646,
 'TXN': -7.02658212784398,
 'ACN': -7.391327222257543,
 'USB': -7.992480308740909,
 'RY.TO': -8.017509945415465,
 'TD.TO': -8.376201531969315,
 'CAT': -8.608348896138697,
 'UNP': -8.768126449825004,
 'PYPL': -10.882981761123762,
 'BB.TO': -16.298227134004076,
 'BMY': -18.90422180297117,
 'T.TO': -19.69571041988972,
 'PFE': -21.281296290827406}

In [28]:
!pip install cvxpy

  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.2
    Uninstalling numpy-1.26.2:
      Successfully uninstalled numpy-1.26.2


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\software\\Anaconda3\\Lib\\site-packages\\~umpy.libs\\libopenblas64__v0.3.23-293-gc2f4bdbb-gcc_10_3_0-2bde3a66a51006b2b53eb373ff767a3f.dll'
Consider using the `--user` option or check the permissions.



In [36]:
import cvxpy as cp


ImportError: DLL load failed while importing lapack_lite: The specified module could not be found.

In [35]:
import cvxpy as cp
def give_weight(num_stocks):
    stocks_now = list(stocks.keys())[:num_stocks]
    prices = np.array(current_data[stocks_now].iloc[0])
    last_values = np.array(current_data[stocks_now].iloc[-1])
    w = cp.Variable(num_stocks)
    my_value = cp.sum(cp.multiply(last_values, w * 1000000 / prices))
    min_weight = (100 / (2 * num_stocks)) / 100
    objective = cp.Minimize(my_value - sp500val)
    constraints = [
        cp.sum(w) == 1,           
        w >= min_weight,                    
        w <= 0.15                      
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    return w.value


ImportError: DLL load failed while importing lapack_lite: The specified module could not be found.

In [ ]:
def get_value(weights,i):
    last_row = current_data[list(stocks.keys())[:i]].iloc[-1]
    first_row = current_data[list(stocks.keys())[:i]].iloc[0]
    return np.sum(np.multiply(last_row,np.divide(weights*1000000,first_row)))

In [ ]:
def stocks_ith_portfolio(numStocks):
    weight_now = give_weight(numStocks)
    curr_val = get_value(weight_now, numStocks)
    return curr_val

In [ ]:
diff = sp500val
close = 0
best_weights = []
best = 0
for i in range(12,25):
    curr_val = stocks_ith_portfolio(i)
    print(curr_val)
    if (abs(average - curr_val) < diff):
        diff = abs(curr_val - average)
        best = i
        best_weights = weight_now
        close = curr_val

In [ ]:
ranked_stocks = stocks

stockslist = list(ranked_stocks.keys())
weightlist = best_weight
weights = dict(zip(stockslist, weightlist))
total_money = 1000000
final_allocation = {}
for stock, weight in weights.items():
    if weight > 0:
        allocation = total_money * weight  
        ticker = yf.Ticker(stock)
        price = ticker.fast_info["lastPrice"]  # need to make sure "end_date" is the most recent trading day
        '''
        currency = ticker.fast_info["currency"]     
        if currency == 'USD':
            exchange = 'USDCAD=X'
            exchange_ticker = yf.Ticker(exchange)
            exchange_rate = exchange_ticker.fast_info["lastPrice"]
            price *= exchange_rate
        elif currency == 'CAD':
            continue
        '''
        num_shares = (allocation / price)
        # considering transaction costs
        if num_shares > 3950:
            num_shares = (allocation - 3.95)/price
            transaction_cost = 3.95
        else:
            num_shares = allocation/(price+0.001)
            transaction_cost = num_shares*0.001
        total_cost = num_shares * price
        #transaction_cost = total_cost * 0.01
        final_allocation[stock] = {
            "Number of Shares": num_shares,
            "Transaction Cost": transaction_cost,
            "Total Money Spent": total_cost + transaction_cost,
            "Cumulative Transaction Cost": np.round(sum(stock["Transaction Cost"] for stock in final_allocation.values()), 2),
            "Cumulative Total": np.round(total_cost + transaction_cost+ sum(stock["Total Money Spent"] for stock in final_allocation.values()), 2)
        }
final = pd.DataFrame(final_allocation)

print(f"Total Money Spent: ${sum(a['Total Money Spent'] for a in final_allocation.values()):.2f}")
print(f"Total Cost: ${sum(a['Transaction Cost'] for a in final_allocation.values()):.2f}")
display(final)

In [ ]:
# put everything into the final portfolio

Portfolio_Final = pd.DataFrame({"Tickers": [], 
                   "Price":[], 
                   "Currency":[], 
                   "Shares":[], 
                   "Value":[], 
                   "Weight":[],
                   "Transactional cost": [],
                   "Cumulative Value": [],
                   "Cumulative Transactional Cost": []})
cumulative_value = 0
cumulative_transaction_cost = 0
for stock, weight in weights.items():
    if weight > 0:
        allocation = total_money * weight  
        ticker = yf.Ticker(stock)
        price = ticker.fast_info["lastPrice"]  # need to make sure "end_date" is the most recent trading day
        num_shares = (allocation / price)
        
        # considering transaction costs
        if num_shares > 3950:
            num_shares = (allocation - 3.95)/price
            transaction_cost = 3.95
        else:
            num_shares = allocation/(price+0.001)
            transaction_cost = num_shares*0.001
        total_cost = num_shares * price

        value = price * num_shares
        cumulative_value += value
        cumulative_transaction_cost += transaction_cost

        new_row = {"Tickers": stock, 
                   "Price": price, 
                   "Currency": "CAD", 
                   "Shares": num_shares, 
                   "Value": value, 
                   "Weight": weight,
                   "Transactional cost": transaction_cost,
                   "Cumulative Value": cumulative_value, 
                   "Cumulative Transactional Cost": cumulative_transaction_cost}
        Portfolio_Final.loc[len(Portfolio_Final)] = new_row 

Portfolio_Final.index = range(1, len(Portfolio_Final) + 1)

Portfolio_Final

In [ ]:
Stocks_Final = pd.DataFrame({"Tickers": Portfolio_Final['Tickers'], 
                             "Shares": Portfolio_Final['Shares']})
display(Stocks_Final)
# output last dataframe as a CSV file
Stocks_Final.to_csv()

In [53]:
diff = sp500val
close = 0
for i in range(12,25):
    weight_now = give_weight(i)
    last_row = current_data[list(stocks.keys())[:i]].iloc[-1]
    first_row = current_data[list(stocks.keys())[:i]].iloc[0]
    curr_val = np.sum(np.multiply(last_row,np.divide(weight_now*1000000,first_row)))
    if (abs(curr_val - sp500val) < diff):
        diff = abs(curr_val - sp500val)
        close = curr_val
print(close)
print(diff)

1177361.3213981315
160709.54860186856


In [54]:
print(diff/close)

0.13649976917113596


In [47]:
sp500val

np.float64(1338070.87)

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

In [ ]:
Abbas, Gurmehar, Elaine